## Imports

In [4]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, classification_report, auc
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

## Preprocessing

In [26]:
# Load data
data = pd.read_csv('/content/Temporal_Features_Data.csv')
data

,id,class_name,@@Pair:A02BC01_ORAL_state:level1@Contain@N02BA_ORAL_state:level2,@@Pair:A02BC01_ORAL_state:level1@Contain@R03BB_RESPIR_state:level1,@@Pair:A02BC01_ORAL_state:level1@Contain@G04CA02_ORAL_state:level1,@@Pair:A02BC01_ORAL_state:level1@Contain@A12AA_ORAL_state:level1,@@Pair:A02BC01_ORAL_state:level1@Contain@M04AA01_ORAL_state:level1,@@Pair:A02BC01_ORAL_state:level1@Contain@C03CA01_ORAL_state:level1,@@Pair:A02BC01_ORAL_state:level1@Contain@R03BB01_RESPIR_state:level1,@@Pair:A02BC01_ORAL_state:level1@Contain@N02AA55_ORAL_state:level1,...,@@Pair:M04AC01_ORAL_state:level1@Contain@M04AA01_ORAL_state:level1,@@Pair:M04AC01_ORAL_state:level1@Contain@N02BA_ORAL_state:level2,@@Pair:M04AC01_ORAL_state:level1@Contain@N02AA_ORAL_state:level1,@@Pair:M04AC01_ORAL_state:level1@Contain@M04AA01_ORAL_state:level3,@@Pair:N02AA_ORAL_state:level1@Contain@N02AA55_ORAL_state:level1,@@Pair:N02BA_ORAL_state:level2@Contain@M04AA01_ORAL_state:level3,@@Pair:N02BA_ORAL_state:level2@Contain@A10BA02_ORAL_state:level2,@@Pair:C07AB03_ORAL_state:level2@Contain@M04AA01_ORAL_state:level1@@Pair:C07AB03_ORAL_state:level2@Contain@N02BA_ORAL_state:level2@@Pair:M04AA01_ORAL_state:level1@Contain@N02BA_ORAL_state:level2,@@Pair:C02CA04_ORAL_state:level1@Contain@C08CA13_ORAL_state:level2@@Pair:C02CA04_ORAL_state:level1@Contain@M04AA01_ORAL_state:level1@@Pair:C08CA13_ORAL_state:level2@Contain@M04AA01_ORAL_state:level1,@@Pair:A10BA02_ORAL_state:level2@Contain@M04AC01_ORAL_state:level1@@Pair:A10BA02_ORAL_state:level2@Contain@N02BA_ORAL_state:level2@@Pair:M04AC01_ORAL_state:level1@Contain@N02BA_ORAL_state:level2
0,4476092,cancer,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
1,4190768,cancer,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
2,3821284,cancer,0.000001,0.000001,0.000001,0.000001,0.142858,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
3,4060888,cancer,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
4,3714187,cancer,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,5413454,neither,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
3104,3957262,neither,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
3105,3442260,neither,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
3106,6027247,neither,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001


# XGboost Model

In [24]:
X = data.drop(['id', 'class_name'], axis=1)
y = data['class_name']

classes = y.unique()
# Convert class labels to categorical codes if they are not numeric
y = pd.Categorical(y).codes

# Splitting the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # Split into train and temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Split temp into validation and test


In [25]:
total_data_count = len(data)  # Total number of entries in the dataset
test_data_count = len(y_test)  # Number of entries in y_test
test_data_percentage = (test_data_count / total_data_count) * 100

print(f"y_test represents {test_data_percentage:.2f}% of the total dataset.")

y_test represents 20.01% of the total dataset.


## Train XGboost Model

In [ ]:
# Initialize the XGBoost classifier
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Define the parameter grid
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.001, 0.01, 0.1],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1]
}

# Set up the grid search with cross-validation
grid_cv = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)
grid_cv.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=True)

best_model = grid_cv.best_estimator_

In [ ]:
# save the model
best_model.save_model('xgb_model.json')

# best_model = XGBClassifier()
# best_model.load_model('xgb_model.json')

## Predict

In [ ]:
# Predict probabilities for each class
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)

## Evaluation

In [ ]:
# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# ROC AUC
roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')
print("ROC AUC (One-vs-Rest):", roc_auc_ovr)

In [ ]:
# Predict probabilities for each class
y_prob = best_model.predict_proba(X_test)

# Binarize the labels for ROC curve plotting
from sklearn.preprocessing import label_binarize
y_test_bin = label_binarize(y_test, classes=np.unique(y_train))

# Setup for plotting
n_classes = y_test_bin.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()

# Calculate dummies once
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'purple', 'orange']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'.format(classes[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for multi-class')
plt.legend(loc="lower right")
plt.show()
